In [ ]:
pip install transformers

In [ ]:
pip install SentencePiece

In [ ]:
pip install pandas

In [ ]:
pip install accelerate -U

In [45]:
import torch
import tensorflow as tf
import pandas as pd
from torch.utils.data import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import Trainer, TrainingArguments
import mysql.connector


In [46]:
class SQLDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.questions = df['question'].tolist()
        self.queries = df['sql'].tolist()
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        query = self.queries[idx]

        inputs = self.tokenizer.encode_plus(
            question,
            padding='max_length',
            truncation=True,
            max_length=128,
            return_tensors='pt'
        )
        labels = self.tokenizer.encode(
            query,
            padding='max_length',
            truncation=True,
            max_length=64,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs.input_ids.squeeze(),
            'attention_mask': inputs.attention_mask.squeeze(),
            'labels': labels.squeeze()
        }

In [47]:
df = pd.read_csv('merged_dataset.csv')

In [48]:
df.head()

,question,sql
0,Tell me what the notes are for South Australia,SELECT Notes FROM table WHERE Current slogan =...
1,What is the current series where the new serie...,SELECT Current series FROM table WHERE Notes =...
2,What is the format for South Australia?,SELECT Format FROM table WHERE State/territory...
3,Name the background colour for the Australian ...,SELECT Text/background colour FROM table WHERE...
4,how many times is the fuel propulsion is cng?,SELECT COUNT Fleet Series (Quantity) FROM tabl...


In [49]:
train_data = df
valid_data = df[40000:45000]
#test_data = df[45000:]

In [53]:
output_dir = './fine-tuned-model'
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained(output_dir)

In [54]:
dataset1 = SQLDataset(train_data, tokenizer)
dataset2 = SQLDataset(valid_data, tokenizer)

In [55]:
dataset1[0]

{'input_ids': tensor([8779,  140,  125,    8, 3358,   33,   21, 1013, 2051,    1,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0

In [56]:
device = torch.device('cuda' if tf.test.is_gpu_available() else 'cpu')

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


In [57]:
tf.test.is_gpu_available()

False

In [58]:
model=model.to(device)
print(model.device)

cpu


In [ ]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./output',
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=1e-5,
    save_total_limit=1,
    #fp16=True,  # Enable mixed-precision training if available
    logging_dir='./logs',
    logging_steps=100,
    save_steps=500,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset1,
    eval_dataset=dataset2,
)


In [ ]:
trainer.train()

In [ ]:
output_dir = './fine-tuned-model'
trainer.save_model(output_dir)

In [ ]:
results = trainer.evaluate(dataset2)
print(results)

In [69]:
input_question = "Find the name,population which has countrycode NLD"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT Name,Population FROM table WHERE Countrycode = NLD</s>


In [60]:
import spacy

def extract_keywords(user_query):

    nlp = spacy.load("en_core_web_sm")

    doc = nlp(user_query)

    keywords = [token.text.lower() for token in doc if token.pos_ in {"NOUN", "PROPN", "ADJ"}]

    return keywords

keywords = extract_keywords(user_query)
print(keywords)


['population', 'district', 'kabol']


In [61]:
def jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0

In [62]:
import mysql.connector
from difflib import SequenceMatcher

def extract_table_name_with_columns(user_query):
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Aman@7221",
        database="world"
    )
    cursor = conn.cursor()

    cursor.execute("SHOW TABLES;")
    table_names = [table[0] for table in cursor.fetchall()]

    table_columns_mapping = {}

    keywords = extract_keywords(user_query)
    print(keywords)
    
    similarity_sums = {}
    for table_name in table_names:
        cursor.execute(f"SHOW COLUMNS FROM {table_name};")
        column_names = [column[0] for column in cursor.fetchall()]
        print(column_names)
        
        # Compute Jaccard similarity between keywords and each column name
        similarities = [jaccard_similarity(set(keywords), set(column_name.lower().split())) for column_name in column_names]
        print(similarities)
        sum_similarity=sum(similarities)
        similarity_sums[table_name]=sum_similarity
        
        # Find the column name with the highest similarity score
        max_similarity_index = similarities.index(max(similarities))
        max_similarity_score = similarities[max_similarity_index]

        threshold = 0.2
        print(threshold)
        if max_similarity_score >= threshold:
            table_columns_mapping[table_name] = column_names[max_similarity_index]
            
    print(table_columns_mapping)
    print("Threshold",similarity_sums)
    
    max_key = max(similarity_sums, key=lambda k: similarity_sums[k])

    print("Identified table name:", max_key)
    print("Associated column name:", table_columns_mapping[max_key])
    return(max_key)

    cursor.close()
    conn.close()

max_key=extract_table_name_with_columns(user_query)


['population', 'district', 'kabol']
['ID', 'Name', 'CountryCode', 'District', 'Population']
[0.0, 0.0, 0.0, 0.3333333333333333, 0.3333333333333333]
0.2
['Code', 'Name', 'Continent', 'Region', 'SurfaceArea', 'IndepYear', 'Population', 'LifeExpectancy', 'GNP', 'GNPOld', 'LocalName', 'GovernmentForm', 'HeadOfState', 'Capital', 'Code2']
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0.2
['CountryCode', 'Language', 'IsOfficial', 'Percentage']
[0.0, 0.0, 0.0, 0.0]
0.2
{'city': 'District', 'country': 'Population'}
Threshold {'city': 0.6666666666666666, 'country': 0.3333333333333333, 'countrylanguage': 0.0}
Identified table name: city
Associated column name: District


In [63]:
def cosine_similarity(vector1, vector2):
    dot_product = sum(v1 * v2 for v1, v2 in zip(vector1, vector2))
    norm1 = math.sqrt(sum(v1 ** 2 for v1 in vector1))
    norm2 = math.sqrt(sum(v2 ** 2 for v2 in vector2))
    if norm1 == 0 or norm2 == 0:
        return 0.0
    
    return dot_product / (norm1 * norm2)

In [64]:
def extract_table_name_with_columns(user_query):
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Aman@7221",
        database="world"
    )
    cursor = conn.cursor()

    cursor.execute("SHOW TABLES;")
    table_names = [table[0] for table in cursor.fetchall()]

    table_columns_mapping = {}

    keywords = extract_keywords(user_query)
    print(keywords)
    
    similarity_sums = {}
    for table_name in table_names:
        cursor.execute(f"SHOW COLUMNS FROM {table_name};")
        column_names = [column[0] for column in cursor.fetchall()]
        print(column_names)
        
        # Compute cosine similarity between keywords and each column name
        vector1 = [1 if any(keyword.lower() in col_name.lower() for col_name in column_names) else 0 for keyword in keywords]
        vector2 = [1 if any(keyword.lower() in col_name.lower() for col_name in column_names) else 0 for keyword in column_names]
        similarity = cosine_similarity(vector1, vector2)
        print(similarity)
        similarity_sums[table_name]=similarity
    
    max_key = max(similarity_sums, key=lambda k: similarity_sums[k])
    print(similarity_sums)
    print("Identified table name:", max_key)
    return max_key

    cursor.close()
    conn.close()

max_key = extract_table_name_with_columns(user_query)


['population', 'district', 'kabol']
['ID', 'Name', 'CountryCode', 'District', 'Population']
0.6324555320336759
['Code', 'Name', 'Continent', 'Region', 'SurfaceArea', 'IndepYear', 'Population', 'LifeExpectancy', 'GNP', 'GNPOld', 'LocalName', 'GovernmentForm', 'HeadOfState', 'Capital', 'Code2']
0.2581988897471611
['CountryCode', 'Language', 'IsOfficial', 'Percentage']
0.0
{'city': 0.6324555320336759, 'country': 0.2581988897471611, 'countrylanguage': 0.0}
Identified table name: city


In [67]:
def change_next_word(text, particular_word, new_word):
    words = text.split()  
    for i, word in enumerate(words[:-1]):  
        if word == particular_word:
            words[i + 1] = new_word 
    return ' '.join(words)

max_key=str(max_key)
modified_text = change_next_word(generated_query, "FROM", max_key)
print(modified_text)


<pad> SELECT Population FROM city WHERE District = kabol</s>
